# Pub/Sub Pull Subscriber

## Create a Subscription and subscribe to the clicks topic

The following example uses a streaming pull to process messages as they arrive. 
The callback function processes the messages and acknowledges reciept. 

In [ ]:
import os
import json
from google.cloud import pubsub_v1
from google.api_core.exceptions import AlreadyExists


# Project, topic, and subscription variables
project_id = 'dsl-dar'
topic_id = 'clicks'
subscription_id = 'my-pull-subscription'

# Initialize the Publisher and Subscriber clients
publisher = pubsub_v1.PublisherClient()
subscriber = pubsub_v1.SubscriberClient()
topic_path = publisher.topic_path(project_id, topic_id)
subscription_path = subscriber.subscription_path(project_id, subscription_id)


# Create the subscription if it doesn't exist
try:
    subscriber.create_subscription(name=subscription_path, topic=topic_path)
    print(f'Subscription {subscription_path} created.')
except AlreadyExists:
    print(f'Subscription {subscription_path} already exists.')

# Callback function to process received messages
def callback(message):
    print(f'Received message: {message.data.decode("utf-8")}')
    message.ack()

# Subscribe to the Pub/Sub topic and pull messages
streaming_pull_future = subscriber.subscribe(subscription_path, callback=callback)
print(f'Listening for messages on {subscription_path}...')

# Wrap subscriber in a 'with' block to automatically call close() when done
with subscriber:
    try:
        # Streaming pull future will block indefinitely
        streaming_pull_future.result()
    except KeyboardInterrupt:
        streaming_pull_future.cancel()
